##### Loading the dataset

In [ ]:
import os, shutil, pathlib

original_dir = pathlib.Path("Data/train")
new_base_dir = pathlib.Path("Data/kaggle_dogs_vs_cats_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

##### EDA: Explore the data with relevant graphs, statistics and insights 

##### importing the required libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pathlib
import matplotlib.image as mpimg

In [ ]:
data_folder = pathlib.Path('Data/kaggle_dogs_vs_cats_small')

##### Random numbers

In [ ]:
random_numbers = np.random.normal(size=(1000, 16))

In [ ]:
print(type(random_numbers))
print(random_numbers.shape)
print(random_numbers.dtype)
print(random_numbers[:4])

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)

In [ ]:
type(dataset)

In [ ]:
for i, d in enumerate(['A','B','C']):
    print(i,d)

In [ ]:
for i, element in enumerate(dataset):
    print(element.shape)
    if i >= 2:
        break

In [ ]:
for i, element in enumerate(dataset):
    print(element)
    if i >= 2:
        break

In [ ]:
batched_dataset = dataset.batch(32)
for i, element in enumerate(batched_dataset):
    print(element.shape)
    if i >= 2:
        break

In [ ]:
type(batched_dataset)

##### Using Keras Utility Functions to Create a Dataset for Images


In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    data_folder / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    data_folder / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    data_folder / "test",
    image_size=(180, 180),
    batch_size=32)

#### Train dataset

In [ ]:
type(train_dataset)

##### Displaying the shapes of the data and labels 

In [ ]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

In [ ]:
labels_batch


In [ ]:
# import imshow
import matplotlib.pyplot as plt

plt.imshow(data_batch[0].numpy().astype("uint8"))

In [ ]:
import random
import matplotlib.image as mpimg
# Preview random 5 images from each class in train set
def plot_sample_images(directory, label, n=5):
    path = os.path.join(directory, label)
    images = random.sample(os.listdir(path), n)
    fig, axes = plt.subplots(1, n, figsize=(15,5))
    for img_name, ax in zip(images, axes):
        img_path = os.path.join(path, img_name)
        img = mpimg.imread(img_path)
        ax.imshow(img)
        ax.set_title(label)
        ax.axis('off')
    plt.tight_layout()

plot_sample_images('Data/kaggle_dogs_vs_cats_small/train','cat')
plot_sample_images('Data/kaggle_dogs_vs_cats_small/train','dog')

##### Training the CNN for a real-world image classification

##### Generating the Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    r'Data\kaggle_dogs_vs_cats_small\train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_gen = val_datagen.flow_from_directory(
    r'Data\kaggle_dogs_vs_cats_small\validation',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


##### Defining the CNN

In [ ]:
model_cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

model_cnn.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

model_cnn.summary()


##### Callbacks and Training the model

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="cnn_best_model.keras",
        save_best_only=True,
        monitor="val_loss"
    ),
]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)


##### Doing the VGG16 and using it as Base

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))

In [ ]:
data_folder = pathlib.Path('Data/kaggle_dogs_vs_cats_small')

train_dataset = image_dataset_from_directory(
    data_folder / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    data_folder / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    data_folder / "test",
    image_size=(180, 180),
    batch_size=32)

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2)
])

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
vgg_model = keras.Model(inputs, outputs)
vgg_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
vgg_model.summary()

In [ ]:
import numpy as np

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels =  get_features_and_labels(train_dataset)
val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)

In [ ]:
callbacks_ft = [
    keras.callbacks.ModelCheckpoint(
        filepath="vgg16_finetune_best.keras",
        save_best_only=True,
        monitor="val_loss"
    ),
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
]

history_vgg = vgg_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks_ft
)

##### Fine tuning the layers

In [ ]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

vgg_model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(1e-5), metrics=["accuracy"])

# Continue training for a few epochs to fine-tune
history_fine = vgg_model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=callbacks_ft
)

##### Model Evaluation and Comparasion

In [ ]:
best_cnn = keras.models.load_model("cnn_best_model.keras")
best_vgg = keras.models.load_model("vgg16_finetune_best.keras")

##### The Accuracy scores are here

In [ ]:
cnn_loss, cnn_acc = best_cnn.evaluate(test_dataset)
vgg_loss, vgg_acc = best_vgg.evaluate(test_dataset)
print(f"Custom CNN Test Accuracy: {cnn_acc:.3f}")
print(f"VGG16 Fine-Tuned Test Accuracy: {vgg_acc:.3f}")

##### Here the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve

# Collect predictions and true labels
y_true = []
y_cnn_pred = []
y_vgg_pred = []
for images, labels in test_dataset:
    y_true.extend(labels.numpy())
    y_cnn_pred.extend(best_cnn.predict(images).flatten())
    y_vgg_pred.extend(best_vgg.predict(images).flatten())
y_true = np.array(y_true)
y_cnn_bin = np.array(y_cnn_pred) > 0.5
y_vgg_bin = np.array(y_vgg_pred) > 0.5

cm_cnn = confusion_matrix(y_true, y_cnn_bin)
cm_vgg = confusion_matrix(y_true, y_vgg_bin)
print("CNN Confusion Matrix:\n", cm_cnn)
print("VGG Confusion Matrix:\n", cm_vgg)

print(f'Custom CNN - Precision: {precision_score(y_true, y_cnn_bin):.2f}, Recall: {recall_score(y_true, y_cnn_bin):.2f}, F1: {f1_score(y_true, y_cnn_bin):.2f}')
print(f'VGG16 Fine-Tuned - Precision: {precision_score(y_true, y_vgg_bin):.2f}, Recall: {recall_score(y_true, y_vgg_bin):.2f}, F1: {f1_score(y_true, y_vgg_bin):.2f}')

##### Visiualizing the Precision-Recall curve 

In [ ]:
pr_cnn = precision_recall_curve(y_true, y_cnn_pred)
pr_vgg = precision_recall_curve(y_true, y_vgg_pred)

plt.plot(pr_cnn[1], pr_cnn[0], label='Custom CNN')
plt.plot(pr_vgg[1], pr_vgg[0], label='VGG16 Fine-Tuned')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

##### Checking the failure analysis 

In [ ]:
import random
failure_idxs = np.where((y_true != y_cnn_bin) | (y_true != y_vgg_bin))[0]
print(f"Number of incorrect predictions: {len(failure_idxs)}")
if len(failure_idxs) > 0:
    ix = random.choice(failure_idxs)
    for batch_images, batch_labels in test_dataset:
        if ix < len(batch_labels):
            img = batch_images[ix].numpy().astype("uint8")
            plt.imshow(img)
            plt.title(
                f"True: {class_names[batch_labels[ix]]}, CNN: {class_names[int(y_cnn_bin[ix])]}, VGG: {class_names[int(y_vgg_bin[ix])]}")
            plt.axis("off")
            plt.show()
            break
        ix -= len(batch_labels)

### Conclusion

#### **Which model performed better? Why?**
The VGG16 transfer learning model outperformed the custom CNN model in both accuracy and loss. This is because it leveraged pre-trained weights from ImageNet, allowing it to extract more complex and generalized features early in training.

#### **How did transfer learning accelerate or improve learning?**
Transfer learning significantly reduced the training time and improved convergence. The pre-trained layers already had a strong understanding of low-level patterns like edges and textures, enabling faster and more accurate learning on the Dogs vs Cats dataset.

#### **Any clear signs of overfitting or underfitting? How did callbacks or augmentation help?**
Some signs of overfitting were observed in the CNN model, especially when validation loss stopped improving. EarlyStopping and ModelCheckpoint helped mitigate overfitting, while data augmentation improved generalization by exposing the model to varied image patterns.

#### **What types of images did both models struggle with?**
Both models struggled with images where dogs and cats appeared in ambiguous poses, low lighting, or with significant occlusions. Misclassifications were common in close-up shots or where animals were partially visible.

#### **What would be the next steps to further boost performance?**
To boost performance, we could fine-tune deeper VGG16 layers, experiment with more powerful models like ResNet or EfficientNet, and optimize hyperparameters. Adding more data or using advanced augmentations could also improve generalization.
